In [ ]:
import os
import shutil

mount_point = '/content/drive'

try:
    # Check if the mount point exists
    if os.path.exists(mount_point):
        # Iterate through all items within the mount point
        for item in os.listdir(mount_point):
            item_path = os.path.join(mount_point, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
                print(f"Removed file: {item_path}")
            elif os.path.isdir(item_path) and item != '.shortcut-targets-by-id':
                shutil.rmtree(item_path)
                print(f"Removed directory: {item_path}")
        print(f"Successfully cleaned the mount point: {mount_point}")
    else:
        print(f"Mount point directory does not exist: {mount_point}")

except Exception as e:
    print(f"An error occurred: {e}")

Removed directory: /content/drive/MyDrive
Successfully cleaned the mount point: /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit langchain langchain_community sentence-transformers ctransformers faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 

In [ ]:
!pip install pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


@st.cache_resource
def load_llm():
    print("Loading the LLM...")
    try:
        llm = CTransformers(
             model="/content/drive/MyDrive/models/mistral",
             model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
             model_type="mistral",
             config={"max_new_tokens": 500, "temperature": 0.01}
         )
        print("LLM loaded successfully!")
        return llm
    except Exception as e:
        print(f"Error loading LLM: {e}")
        st.error(f"Error loading the language model. Please check your internet connection and try again.")
        return None


@st.cache_resource(show_spinner=False)
def setup_conversational_qa(_llm, _vect):
    prompt_template = """You are a helpful chatbot for a bank. Use the following context to answer the customer's question.
    If the context does not contain the answer, truthfully say "I don't know". Do not try to make up an answer.
    Provide concise and accurate information.

    Context: {context}
    Question: {question}
    Answer: """
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    retriever = _vect.as_retriever(search_kwargs={'k': 2})
    qa = ConversationalRetrievalChain.from_llm(
        _llm,
        retriever=retriever,
        memory=memory,
        verbose=False,
        combine_docs_chain_kwargs={"prompt": PROMPT}
    )
    return qa

# --- Main Streamlit App ---
def main():
    st.title("🏦 Bank Customer Support Chatbot")

      # Load existing vector store
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                        model_kwargs={"device": "cpu"})
    vector_store = FAISS.load_local("/content/drive/MyDrive/db/faiss", embedding, allow_dangerous_deserialization=True)

    llm = load_llm()
    qa_chain = setup_conversational_qa(llm, vector_store)

    # Initialize chat history in Streamlit session state
    if "messages" not in st.session_state:
        st.session_state["messages"] = [{"role": "assistant", "content": "How can I help you today?"}]

    # Display chat messages from history on app rerun
    for message in st.session_state["messages"]:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # React to user input
    if prompt := st.chat_input("Ask your banking question here..."):
        # Check if the user wants to end the chat
        end_chat_phrases = ["bye", "goodbye", "end chat", "finish", "quit", "exit", "thank you", "thanks"]
        if any(phrase in prompt.lower() for phrase in end_chat_phrases):
            st.session_state["messages"] = [{"role": "assistant", "content": "You're welcome! If you have more questions in the future, feel free to ask."}]
            st.rerun() # Force a rerun to display the new message and clear the input
        else:
            # Display user message in chat message container
            st.session_state["messages"].append({"role": "user", "content": prompt})
            with st.chat_message("user"):
                st.markdown(prompt)

        # Generate and display assistant response in chat message container
        with st.chat_message("assistant"):
            with st.spinner("Thinking..."):
                response = qa_chain({"question": prompt})
                st.markdown(response["answer"])
                st.session_state["messages"].append({"role": "assistant", "content": response["answer"]})

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠧

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.9.165.142
⠙your url is: https://great-days-mate.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:8733 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙